# Coder


## Import


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import ast
import os
from pathlib import Path
from typing import Any

from haystack import Document, Pipeline, component

# Note: the following requires a "pip install sentence-transformers"
from haystack.components.embedders import (
    SentenceTransformersDocumentEmbedder,
    SentenceTransformersTextEmbedder,
)
from haystack.components.writers import DocumentWriter
from haystack.dataclasses import Document
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from loguru import logger as lg
from rich import print as rprint
from sentence_transformers import SentenceTransformer

from coder.config.coder_config import CoderConfig

## Setup config


In [ ]:
cc = CoderConfig()
rprint(cc.paths)

## Ingest data


In [5]:
class CodeFileDocumentExtractor:
    """
    Tool class that processes a single Python file and splits the code into several chunks.
    Chunks are based on:
      - Top-level functions,
      - Top-level classes,
      - Methods inside classes,
      - And code outside these definitions (global code).

    For each chunk, the following metadata is saved:
      - 'chunk_number': Sequential number of the chunk.
      - 'first_line': The first line of code in the chunk.
      - 'start_line' and 'end_line': The boundaries in the original file.
      - 'type': Type of the chunk (e.g. 'global', 'function', 'class', 'method').
    """

    def __init__(self, file_path: Path):
        self.file_path = file_path

    def _parse_code(self, code: str) -> ast.AST:
        """Parses the source code into an AST."""
        return ast.parse(code)

    def _get_top_level_defs(
        self, tree: ast.AST
    ) -> list[tuple[int, int, str, str, ast.AST]]:
        """
        Returns a list of top-level definitions as tuples:
        (start_line, end_line, type, name, node)
        """
        defs = []
        for node in tree.body:
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                defs.append(
                    (
                        node.lineno,
                        getattr(node, "end_lineno", node.lineno),
                        "function",
                        node.name,
                        node,
                    )
                )
            elif isinstance(node, ast.ClassDef):
                defs.append(
                    (
                        node.lineno,
                        getattr(node, "end_lineno", node.lineno),
                        "class",
                        node.name,
                        node,
                    )
                )
        return sorted(defs, key=lambda x: x[0])

    def _extract_global_intervals(
        self,
        code_lines: list[str],
        top_level_defs: list[tuple[int, int, str, str, ast.AST]],
    ) -> list[tuple[int, int, dict[str, Any]]]:
        """
        Determines intervals (line ranges) for global code segments that are not covered by any
        top-level definition.
        """
        intervals = []
        last_end = 1
        for start, end, typ, name, node in top_level_defs:
            if start > last_end:
                intervals.append((last_end, start - 1, {"type": "global"}))
            last_end = max(last_end, end + 1)
        if last_end <= len(code_lines):
            intervals.append((last_end, len(code_lines), {"type": "global"}))
        return intervals

    def _process_function(
        self, start: int, end: int, name: str
    ) -> list[tuple[int, int, dict[str, Any]]]:
        """Returns the interval for a function."""
        return [(start, end, {"type": "function", "name": name})]

    def _process_class(
        self, class_node: ast.ClassDef
    ) -> list[tuple[int, int, dict[str, Any]]]:
        """
        Processes a class node, splitting it into:
          - Header (code before the first method),
          - Method intervals,
          - Gaps between methods (middle), and
          - Footer (code after the last method).
        """
        intervals = []
        class_start = class_node.lineno
        class_end = getattr(class_node, "end_lineno", class_node.lineno)
        methods = []
        for child in class_node.body:
            if isinstance(child, (ast.FunctionDef, ast.AsyncFunctionDef)):
                methods.append(
                    (
                        child.lineno,
                        getattr(child, "end_lineno", child.lineno),
                        "method",
                        child.name,
                    )
                )
        methods.sort(key=lambda x: x[0])
        if methods:
            # Class header: before the first method.
            if class_start < methods[0][0]:
                intervals.append(
                    (
                        class_start,
                        methods[0][0] - 1,
                        {"type": "class", "name": class_node.name, "part": "header"},
                    )
                )
            prev_end = methods[0][1]
            intervals.append(
                (
                    methods[0][0],
                    methods[0][1],
                    {"type": "method", "class": class_node.name, "name": methods[0][3]},
                )
            )
            for i in range(1, len(methods)):
                # Gap between methods.
                if prev_end + 1 < methods[i][0]:
                    intervals.append(
                        (
                            prev_end + 1,
                            methods[i][0] - 1,
                            {
                                "type": "class",
                                "name": class_node.name,
                                "part": "middle",
                            },
                        )
                    )
                intervals.append(
                    (
                        methods[i][0],
                        methods[i][1],
                        {
                            "type": "method",
                            "class": class_node.name,
                            "name": methods[i][3],
                        },
                    )
                )
                prev_end = methods[i][1]
            # Class footer: after the last method.
            if prev_end < class_end:
                intervals.append(
                    (
                        prev_end + 1,
                        class_end,
                        {"type": "class", "name": class_node.name, "part": "footer"},
                    )
                )
        else:
            intervals.append(
                (class_start, class_end, {"type": "class", "name": class_node.name})
            )
        return intervals

    def _extract_defs_intervals(
        self, tree: ast.AST
    ) -> list[tuple[int, int, dict[str, Any]]]:
        """
        Processes all top-level definitions (functions and classes) and returns their intervals.
        """
        intervals = []
        top_level_defs = self._get_top_level_defs(tree)
        for start, end, typ, name, node in top_level_defs:
            if typ == "function":
                intervals.extend(self._process_function(start, end, name))
            elif typ == "class":
                intervals.extend(self._process_class(node))
        return intervals

    def _create_documents_from_intervals(
        self, code_lines: list[str], intervals: list[tuple[int, int, dict[str, Any]]]
    ) -> list[Document]:
        """
        Converts each interval (chunk) into a Haystack Document with rich metadata.
        """
        documents = []
        chunk_number = 1
        for start, end, meta in intervals:
            if start > end or start < 1 or end > len(code_lines):
                continue
            chunk_lines = code_lines[start - 1 : end]  # Convert to 0-indexed.
            content = "\n".join(chunk_lines)
            first_line = chunk_lines[0] if chunk_lines else ""
            meta_updated = meta.copy()
            meta_updated.update(
                {
                    "chunk_number": chunk_number,
                    "first_line": first_line,
                    "start_line": start,
                    "end_line": end,
                    "file_path": str(self.file_path),
                }
            )
            # skip empty chunks
            if not content.strip():
                lines = f"lines {start}-{end}"
                lg.debug(f"Skipping empty chunk {chunk_number} ({lines})")
                # chunk_number += 1
                continue
            documents.append(Document(content=content, meta=meta_updated))
            chunk_number += 1
        return documents

    def extract_documents(self) -> list[Document]:
        """
        Orchestrates the extraction process:
          1. Reads the file and splits it into lines.
          2. Parses the code into an AST.
          3. Extracts intervals for definitions and global code.
          4. Converts intervals into Haystack Documents.
        """
        code = self.file_path.read_text(encoding="utf-8")
        code_lines = code.splitlines()
        try:
            tree = self._parse_code(code)
        except Exception as e:
            lg.warning(f"Failed to parse {self.file_path}: {e}")
            return []

        # Get intervals for definitions and global code.
        defs_intervals = self._extract_defs_intervals(tree)
        top_level_defs = self._get_top_level_defs(tree)
        global_intervals = self._extract_global_intervals(code_lines, top_level_defs)
        intervals = defs_intervals + global_intervals
        intervals.sort(key=lambda x: (x[0], x[1]))

        # Create documents from intervals.
        documents = self._create_documents_from_intervals(code_lines, intervals)
        return documents

In [6]:
@component
class CodeFolderIngestor:
    """
    Pipeline component that accepts a list of folder paths, uses the CodeFileDocumentExtractor
    for each Python file found in each folder, and aggregates the resulting Haystack Documents.
    """

    @component.output_types(documents=list[Document])
    def run(self, folder_paths: list[Path]) -> dict[str, list[Document]]:
        documents: list[Document] = []
        for folder_path in folder_paths:
            for file_path in folder_path.rglob("*.py"):
                extractor = CodeFileDocumentExtractor(file_path)
                document = extractor.extract_documents()
                documents.extend(document)
        return {"documents": documents}

In [ ]:
folders = [cc.paths.src_fol]
ingestor = CodeFolderIngestor()
result = ingestor.run(folders)
documents = result["documents"]
print(f"Ingested {len(documents)} documents from the provided folder paths.")

In [ ]:
documents[0].content[:100], documents[2].meta

## Load vdb


In [9]:
document_store = ChromaDocumentStore(
    # persist_path=str(cc.paths.chroma_fol),
)

# document_store.write_documents(documents=documents)
# print(document_store.count_documents())

In [ ]:
indexing = Pipeline()
indexing.add_component("embedder", SentenceTransformersDocumentEmbedder())
indexing.add_component("writer", DocumentWriter(document_store))
indexing.connect("embedder.documents", "writer.documents")
indexing.run({"embedder": {"documents": documents}})
print(document_store.count_documents())

In [ ]:
# sample retriever

retriever = ChromaEmbeddingRetriever(
    document_store=document_store,
    top_k=3,
)

# example run query
retriever.run(query_embedding=[0.1] * 768)

### Sample retrieve embedding the query


In [ ]:
querying = Pipeline()
querying.add_component("query_embedder", SentenceTransformersTextEmbedder())
querying.add_component("retriever", ChromaEmbeddingRetriever(document_store))
querying.connect("query_embedder.embedding", "retriever.query_embedding")
results = querying.run({"query_embedder": {"text": "Variable declarations"}})

for d in results["retriever"]["documents"]:
    print(d.meta, d.score)

## Retriever expanded context


In [20]:
# from typing import List, Dict, Any, Set, Tuple
# from haystack import component
# from haystack import Document


@component
class ContextRetriever:
    """
    Retriever component that enhances a list of retrieved documents by fetching
    additional context documents from a Chroma store. The context is determined based
    on the 'chunk_number' and 'file_path' metadata, retrieving documents whose
    chunk_number is within a specified window before and after the original chunk.
    """

    def __init__(
        self,
        chroma_store,
        context_window_before: int = 1,
        context_window_after: int = 3,
    ) -> None:
        """
        :param chroma_store: An instance of a Chroma document store that supports filter_documents.
        :param context_window: Number of chunks to retrieve before and after the original chunk.
        """
        self.chroma_store = chroma_store
        self.context_window_before = context_window_before
        self.context_window_after = context_window_after

    @component.output_types(documents=list[Document])
    def run(self, retrieved_docs: list[Document]) -> dict[str, list[Document]]:
        """
        For each document in retrieved_docs, query the Chroma store for additional context
        documents based on the file_path and chunk_number metadata.

        :param retrieved_docs: List of documents initially retrieved by the RAG pipeline.
        :return: A dictionary with key 'documents' mapping to the list of context-enhanced documents.
        """
        context_documents: list[Document] = []
        seen: set[tuple[Any, Any]] = set()  # To avoid duplicate documents

        for doc in retrieved_docs:
            file_path = doc.meta.get("file_path")
            chunk_number = doc.meta.get("chunk_number")
            if file_path is None or chunk_number is None:
                continue  # Skip documents missing essential metadata

            # Define a filter to retrieve documents with a chunk_number within the context window

            filters = {
                "operator": "AND",
                "conditions": [
                    {"field": "meta.file_path", "operator": "==", "value": file_path},
                    {
                        "field": "meta.chunk_number",
                        "operator": ">=",
                        "value": chunk_number - self.context_window_before,
                    },
                    {
                        "field": "meta.chunk_number",
                        "operator": "<=",
                        "value": chunk_number + self.context_window_after,
                    },
                ],
            }

            # Retrieve context documents from the Chroma store
            docs = self.chroma_store.filter_documents(filters=filters)
            for d in docs:
                key = (d.meta.get("file_path"), d.meta.get("chunk_number"))
                if key not in seen:
                    seen.add(key)
                    context_documents.append(d)

        return {"documents": context_documents}

In [ ]:
querying = Pipeline()
querying.add_component("query_embedder", SentenceTransformersTextEmbedder())
querying.add_component("retriever", ChromaEmbeddingRetriever(document_store))
querying.connect("query_embedder.embedding", "retriever.query_embedding")
results = querying.run({"query_embedder": {"text": "Variable declarations"}})

for d in results["retriever"]["documents"]:
    print(d.meta, d.score)

In [ ]:
base_retriever = ChromaEmbeddingRetriever(
    document_store,
    top_k=3,
)

# Pass the same document store (or its chroma store) to the ContextRetriever
context_retriever = ContextRetriever(
    chroma_store=document_store,
    context_window_before=1,
    context_window_after=3,
)

# Build the pipeline
querying = Pipeline()
querying.add_component("query_embedder", SentenceTransformersTextEmbedder())
querying.add_component("retriever", base_retriever)
querying.add_component("context_retriever", context_retriever)

# Connect components:
# Connect the query embedder to the retriever
querying.connect("query_embedder.embedding", "retriever.query_embedding")
# Connect the output documents of the retriever to the context retriever
querying.connect("retriever.documents", "context_retriever.retrieved_docs")

# Run the pipeline
results = querying.run(
    {"query_embedder": {"text": "Variable declarations"}},
    include_outputs_from={"retriever", "context_retriever"},
)

# Retrieve original and expanded context documents
original_docs = results["retriever"]["documents"]
expanded_context_docs = results["context_retriever"]["documents"]

# For debugging: print both sets
print("Original Documents:")
for d in original_docs:
    print(d.meta, d.score)

print("\nExpanded Context Documents:")
for d in expanded_context_docs:
    print(d.meta, d.score)

# Dump
